# Option Pricing: Black-Scholes, Heston, and Merton
## Overall Objective: Understand and compare three fundamental option pricing models.

<span style="color:gray; opacity: 0.5;">**1. Black-Scholes:** The benchmark model assuming constant volatility.</span>  
    <span style="color:gray; opacity: 0.5;">_Provides a great baseline and is computationally efficient, but assumes constant $\sigma$ which is unrealistic for modern markets._</span>

<span style="color:gray; opacity: 0.5;">**2. Merton Jump:** Adds "jumps" to the asset price to model market shocks.</span>  
    <span style="color:gray; opacity: 0.5;">_Captures "Fat Tails" and sudden crashes via Poisson jumps._</span>

    
<div style="border-left: 4px solid #2196F3; padding-left: 15px;">

**3. Heston: Adds stochastic volatility (volatility clustering and mean reversion).**   
_Captures the "Smirk" or "Skew" via stochastic vol—essential for pricing OTM puts accurately._
</div>





## 3. The Heston Model (Stochastic Volatility)

Black-Scholes fails to capture the "volatility smile" observed in real markets. Heston solves this by modeling variance $v_t$ as a stochastic process itself.

### The Dynamics (SDEs)

The Heston model uses two coupled Stochastic Differential Equations (SDEs):

1. **Asset Process:**
    
    $$dS_t = \mu S_t dt + \sqrt{v_t} S_t dW_t^S$$
    
2. **Variance Process (CIR Process):**
    
    $$dv_t = \kappa (\theta - v_t) dt + \xi \sqrt{v_t} dW_t^v$$

**Parameters:**

- $v_0$: Initial variance.
    
- $\kappa$: Mean reversion speed (how fast vol returns to average).
    
- $\theta$: Long-run average variance.
    
- $\xi$: "Vol of Vol" (volatility of the variance).
    
- $\rho$: Correlation between asset returns and variance shocks ($dW_t^S$ and $dW_t^v$). usually $\rho < 0$.

<br>

### Pricing via Characteristic Functions

Since there is no simple closed-form solution like Black-Scholes, we use the Fourier Transform method. The price is an integral of the Characteristic Function $\phi$.

**Important Note on Stability (Albrecher et al.):**

Standard implementations of the Heston characteristic function often suffer from "branch cut" discontinuities, leading to numerical explosions (sawtooth graphs). We use the "Albrecher" representation to ensure the characteristic function remains continuous and stable.

## Imports and Setup

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as si
from scipy.integrate import quad
import seaborn as sns

# Set plotting style
sns.set_style("darkgrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Global Parameters (Toggles)
S0 = 100.0    # Spot Price
K_list = np.linspace(80, 120, 50) # Range of Strikes for plotting
T = 1.0       # Time to Maturity (1 year)
r = 0.05      # Risk-free rate
q = 0.0       # Dividend yield

In [3]:
# Heston Parameters
v0 = 0.04    # Current Variance (equiv to 20% vol)
kappa = 2.0  # Mean reversion speed
theta = 0.04 # Long run average variance
xi = 0.3     # Vol of Vol
rho = -0.7   # Correlation

# Calculate Prices
heston_prices = [heston_call_price(S0, k, T, r, q, v0, kappa, theta, xi, rho) for k in K_list]
bs_prices = [black_scholes_call(S0, k, T, r, q, np.sqrt(v0)) for k in K_list]

# Visualization
plt.figure(figsize=(10, 6))
plt.plot(K_list, bs_prices, 'b--', label='Black-Scholes (Vol=20%)')
plt.plot(K_list, heston_prices, 'r-', label='Heston Model')
plt.xlabel('Strike Price')
plt.ylabel('Option Price')
plt.title('Sanity Check: Heston vs Black-Scholes')
plt.legend()

# Difference Plot (to see the "Smile" effect implicitly)
plt.figure(figsize=(10, 4))
plt.plot(K_list, np.array(heston_prices) - np.array(bs_prices), 'g-')
plt.axhline(0, color='black', lw=1)
plt.title('Price Difference (Heston - BS): The Value of Skew')
plt.xlabel('Strike Price')
plt.ylabel('Difference')
plt.show()

NameError: name 'heston_call_price' is not defined

## Heston Greeks (Finite Differences)

Since Heston derivatives are complex to derive analytically, we use **Finite Differences**. This approximates the slope of the pricing surface by nudging the inputs slightly.

We use **Central Difference** for better accuracy:

$$\Delta \approx \frac{P(S + \epsilon) - P(S - \epsilon)}{2\epsilon}$$

$$\Gamma \approx \frac{P(S + \epsilon) - 2P(S) + P(S - \epsilon)}{\epsilon^2}$$

- **Delta ($\Delta$):** Sensitivity to Spot Price.
    
- **Gamma ($\Gamma$):** Sensitivity to Delta (curvature).
    
- **Vega ($\nu$):** Sensitivity to Volatility (in Heston, sensitivity to initial variance $v_0$).
    
- **Theta ($\Theta$):** Sensitivity to Time Decay.
    

In [4]:
def heston_greeks(S0, K, T, r, q, v0, kappa, theta, xi, rho):
    """
    Calculates Heston Greeks using Central Finite Differences.
    Returns a dictionary of Greeks.
    """
    # Perturbations
    dS = S0 * 0.01  # 1% spot shift
    dv = v0 * 0.01  # 1% variance shift
    dt = 1 / 365    # 1 day shift
    
    # Base Price (Center)
    p_base = heston_call_price(S0, K, T, r, q, v0, kappa, theta, xi, rho)
    
    # --- Delta & Gamma (Shock Spot) ---
    p_up = heston_call_price(S0 + dS, K, T, r, q, v0, kappa, theta, xi, rho)
    p_down = heston_call_price(S0 - dS, K, T, r, q, v0, kappa, theta, xi, rho)
    
    delta = (p_up - p_down) / (2 * dS)
    gamma = (p_up - 2 * p_base + p_down) / (dS ** 2)
    
    # --- Vega (Shock Initial Variance v0) ---
    # Note: Vega in Heston is dP/d(sqrt(v0)) usually, but here we do dP/dv0 for strictly variance sensitivity
    # To make it comparable to BS Vega, we often normalize or just track dP/dv0
    p_v_up = heston_call_price(S0, K, T, r, q, v0 + dv, kappa, theta, xi, rho)
    p_v_down = heston_call_price(S0, K, T, r, q, v0 - dv, kappa, theta, xi, rho)
    vega = (p_v_up - p_v_down) / (2 * dv) 
    
    # --- Theta (Shock Time) ---
    # Theta is usually negative (decay), so we look at Price(T - dt) - Price(T)
    # But for the formula T is "time to maturity", so as time passes, T decreases.
    p_t_minus = heston_call_price(S0, K, T - dt, r, q, v0, kappa, theta, xi, rho)
    theta_val = (p_t_minus - p_base) / dt 
    
    return {
        "Price": p_base,
        "Delta": delta,
        "Gamma": gamma,
        "Vega_v0": vega,
        "Theta": theta_val
    }

# --- Test the Greeks ---
greeks = heston_greeks(S0=100, K=100, T=1.0, r=0.05, q=0.0, v0=0.04, 
                       kappa=2.0, theta=0.04, xi=0.3, rho=-0.7)

print(f"Heston ATM Call Greeks:\n{pd.Series(greeks)}")

NameError: name 'heston_call_price' is not defined